In [1]:
%load_ext sql
%sql sqlite:///store.db

## Задание 1
Для каждого артиста вывести все жанры, которые есть в его песнях, и для каждого жанра вывести наиболее продолжительную песню этого артиста.

In [2]:
%%sql

WITH artist_genres AS (
  SELECT a1.artistid, a1.name artist, g.genreid, g.name genre
  FROM artists a1
  CROSS JOIN genres g
  WHERE g.genreid IN (
    SELECT genreid
    FROM tracks
    JOIN albums USING(albumid)
    JOIN artists a2 USING(artistid)
    WHERE a1.artistid = a2.artistid
  )
  ORDER BY a1.artistid
),
longest_songs AS (
  SELECT artistid, artist, genreid, genre, t.name track,
  MAX(milliseconds) OVER(PARTITION BY genreid, artistid) duration
  FROM tracks t
  JOIN albums USING(albumid)
  JOIN artist_genres ag USING(artistid, genreid)
)
SELECT artistid, artist, genre, track, duration
FROM longest_songs
GROUP BY artistid, genre
ORDER BY artistid, genre
LIMIT 10;

 * sqlite:///store.db
Done.


artistid,artist,genre,track,duration
1,AC/DC,Rock,For Those About To Rock (We Salute You),369319
2,Accept,Rock,Balls to the Wall,375418
3,Aerosmith,Rock,Walk On Water,381231
4,Alanis Morissette,Rock,All I Really Want,491885
5,Alice In Chains,Rock,We Die Young,387134
6,Antônio Carlos Jobim,Jazz,Desafinado,366837
6,Antônio Carlos Jobim,Latin,Garota De Ipanema,389642
7,Apocalyptica,Metal,Enter Sandman,436453
8,Audioslave,Alternative,Revelations,301974
8,Audioslave,Alternative & Punk,Your Time Has Come,309786


## Задание 2
Посчитать скользящее среднее выручки по годам.


In [3]:
%%sql

WITH corrected_invoices AS (
    SELECT invoiceid, invoicedate,
    ii.quantity * t.unitprice total
    FROM invoices i
    JOIN invoice_items ii USING(invoiceid)
    JOIN tracks t USING(trackid)
    GROUP BY invoiceid
),
yearly_invoices AS (
    SELECT 
        invoiceid,
        invoicedate,
        SUM(total) total,
        STRFTIME('%m', invoicedate) AS month,
        STRFTIME('%Y', invoicedate) AS year
    FROM corrected_invoices
    GROUP BY year, month
)
SELECT year, month,
SUM(total) OVER(
    PARTITION BY year, month
    ORDER BY invoicedate
    ROWS BETWEEN 11 PRECEDING AND CURRENT ROW
) rolling_avg
FROM yearly_invoices
ORDER BY year, month;

 * sqlite:///store.db
Done.


year,month,rolling_avg
2023,01,48.68
2023,02,44.730000000000004
2023,03,55.55
2023,04,31.689999999999998
2023,05,55.62
2023,06,24.32
2023,07,62.14
2023,08,61.59
2023,09,51.09
2023,10,49.629999999999995


## Задание 3

Для каждого плейлиста вывести трек с наибольшей продолжительностью, трек с наименьшей стоимостью (если у треков одинаковая стоимость, то вывести тот, который весит меньше всех), и жанр данного плейлиста (жанром плейлиста будет называться тот жанр, треков которого больше всех в плейлисте).

In [4]:
%%sql

WITH playlist_tracks AS (
    SELECT t.*, playlistid, p.name playlist, g.name genre
    FROM tracks t
    JOIN playlist_track USING(trackid)
    JOIN playlists p USING(playlistid)
    JOIN genres g USING(genreid)
),
intermediate_query AS (
    SELECT DISTINCT(playlistid), playlist,
    MAX(milliseconds) OVER (PARTITION BY playlistid) max_duration,
    MIN(unitprice) OVER (PARTITION BY playlistid) min_price
    FROM playlist_tracks
)
SELECT playlistid, playlist,
(
    SELECT name
    FROM playlist_tracks t
    WHERE t.playlistid = iq.playlistid
    AND t.milliseconds = iq.max_duration
) max_duration_track,
(
    SELECT name
    FROM playlist_tracks t
    WHERE t.playlistid = iq.playlistid
    AND t.unitprice = iq.min_price
    AND bytes = (
        SELECT MIN(bytes)
        FROM playlist_tracks t
        WHERE t.playlistid = iq.playlistid
        AND t.unitprice = iq.min_price
    )
) min_price_track,
(
    SELECT genre
    FROM playlist_tracks pt
    WHERE playlistid = iq.playlistid
    GROUP BY genreid
    ORDER BY COUNT(*) DESC
    LIMIT 1
) most_occuring_genre
FROM intermediate_query iq;

 * sqlite:///store.db
Done.


playlistid,playlist,max_duration_track,min_price_track,most_occuring_genre
3,From That TV Show,Occupation / Precipice,Your Time Is Gonna Come,TV Shows
4,Romantic Collection 87,Advance Romance,"Oh, My Love",Rock
5,90’s Music,Coma,Perfect Crime,Metal
10,Hidden Gems,Occupation / Precipice,Greatest Hits,TV Shows
11,Brazilian Music,Vai Passar,A Banda,Latin
13,Classical,"Symphony No. 3 Op. 36 for Orchestra and Soprano ""Symfonia Piesni Zalosnych"" \ Lento E Largo - Tranquillissimo","L'orfeo, Act 3, Sinfonia (Orchestra)",Classical
15,The Basics Of Classic,"Adagio for Strings from the String Quartet, Op. 11","Aria Mit 30 Veränderungen, BWV 988 ""Goldberg Variations"": Aria",Classical
16,Your Sisters' Favorite Tracks,Alive,American Idiot,Rock
17,Rock Classic,Master Of Puppets,Run to the Hills,Metal


In [5]:
%%sql
SELECT * FROM playlists;

 * sqlite:///store.db
Done.


PlaylistId,Name
3,From That TV Show
4,Romantic Collection 87
5,90’s Music
6,Guitar Classic
10,Hidden Gems
11,Brazilian Music
13,Classical
15,The Basics Of Classic
16,Your Sisters' Favorite Tracks
17,Rock Classic


## Задание 4

Для каждого жанра вывести треки, которые встречались в большем числе заказов,
сумму, потраченную на данные треки, количество купленных треков и компанию, которая больше всех приобрела данную композицию.

In [6]:
%%sql

WITH joined_tables AS (
    SELECT 
        t.trackid,
        t.name track,
        t.unitprice price,
        t.genreid,
        g.name genre,
        c.company,
        SUM(t.unitprice * ii.quantity) OVER(PARTITION BY trackid) total_spent,
        SUM(ii.quantity) OVER(PARTITION BY trackid) total_bought
    FROM tracks t
    JOIN genres g USING(genreid)
    JOIN invoice_items ii USING(trackid)
    JOIN invoices i USING(invoiceid)
    JOIN customers c USING(customerid)
),
most_occuring_track_query AS (
    SELECT *,
    (
        SELECT trackid
        FROM joined_tables jt_inner
        WHERE jt.genreid = jt_inner.genreid
        GROUP BY trackid
        ORDER BY COUNT(*) DESC
        LIMIT 1
    ) most_occuring_trackid
    FROM joined_tables jt
),
most_buying_company_query AS (
    SELECT *,
    (
        SELECT company
        FROM most_occuring_track_query motq_inner
        WHERE motq.genreid = motq_inner.genreid AND company IS NOT NULL
        GROUP BY company
        ORDER BY COUNT(*) DESC
        LIMIT 1
    ) most_buying_company
    FROM most_occuring_track_query motq
    WHERE trackid = motq.most_occuring_trackid
    GROUP BY genreid
)
SELECT genreid, genre, track most_occuring_track, total_spent, total_bought, most_buying_company
FROM most_buying_company_query
ORDER BY genreid;

 * sqlite:///store.db


Done.


genreid,genre,most_occuring_track,total_spent,total_bought,most_buying_company
1,Rock,War Pigs,3.96,4,Google Inc.
2,Jazz,End Of Romanticism,2.9699999999999998,3,Microsoft Corporation
3,Metal,Question!,8.97,3,Google Inc.
4,Alternative & Punk,Toda Cor,5.9399999999999995,6,Microsoft Corporation
5,Rock And Roll,Good Golly Miss Molly,3.98,2,Google Inc.
6,Blues,Title Song,1.98,2,BMW
7,Latin,Não Quero Dinheiro,1.98,2,Google Inc.
8,Reggae,C'est La Vie,1.98,2,Microsoft Corporation
9,Pop,Beautiful Boy,1.98,2,JetBrains s.r.o.
10,Soundtrack,Plot 180,2.9699999999999998,3,Microsoft Corporation


## Задание 5
Составить рейтинг клиентов по количеству потраченных ими денег.

In [7]:
%%sql

WITH corrected_invoices AS (
    SELECT invoiceid, customerid,
    ii.quantity * t.unitprice total
    FROM invoices i
    JOIN invoice_items ii USING(invoiceid)
    JOIN tracks t USING(trackid)
    GROUP BY invoiceid
),
joined_tables AS (
    SELECT
        c.customerid,
        c.firstname || " " || c.lastname customer,
        SUM(i.total) total_spent
    FROM customers c
    JOIN corrected_invoices i USING(customerid)
    GROUP BY customerid
)
SELECT *, RANK() OVER(ORDER BY total_spent DESC) rank
FROM joined_tables LIMIT 10;

 * sqlite:///store.db
Done.


customerid,customer,total_spent,rank
49,Stanisław Wójcik,30.85,1
17,Jack Smith,24.89,2
42,Wyatt Girard,21.37,3
14,Mark Philips,19.88,4
3,François Tremblay,19.39,5
4,Bjørn Hansen,19.37,6
40,Dominique Lefebvre,18.89,7
55,Mark Taylor,18.88,8
24,Frank Ralston,17.900000000000002,9
15,Jennifer Peterson,17.89,10
